# 基于Transformers的阅读理解实现-滑窗处理

## step1 导入包

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator

d:\anaconda3\envs\py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## step2 加载数据集

In [2]:
datasets = load_dataset('cmrc2018', cache_dir='./data')
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

In [3]:
print(datasets['train'][0])

{'id': 'TRAIN_186_QUERY_0', 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。', 'question': '范廷颂是什么时候被任为主教的？', 'answers': {'text': ['1963年'], 'answer_start': [30]}}


## step3 数据集预处理

In [3]:
tokenizer = AutoTokenizer.from_pretrained('D:/pretrained_model/models--hfl--chinese-macbert-base')

NameError: name 'AutoTokenizer' is not defined

In [5]:
print(datasets['train'][0])

{'id': 'TRAIN_186_QUERY_0', 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。', 'question': '范廷颂是什么时候被任为主教的？', 'answers': {'text': ['1963年'], 'answer_start': [30]}}


In [6]:
sample_dataset = datasets['train'].select(range(2))
sample_dataset

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 2
})

In [2]:
tokenizer_examples = tokenizer(text=sample_dataset['question'],
                               text_pair=sample_dataset['context'],
                               max_length=512,
                               truncation='only_second',
                               return_offsets_mapping=True,
                               return_overflowing_tokens = True,
                               stride=128,
                               padding='max_length')

NameError: name 'tokenizer' is not defined

In [ ]:
tokenizer_examples.keys()

In [ ]:
print(tokenizer_examples['offset_mapping'][0], len(tokenizer_examples['offset_mapping'][0]))

In [ ]:
offset_mapping = tokenizer_examples.pop("offset_mapping")

In [ ]:
print(tokenizer_examples)

In [ ]:
tokenizer_examples.sequence_ids(0)

In [ ]:
offset_mapping, len(offset_mapping)

In [ ]:
# 处理截断的过程中，答案被截断的情况

for idx, offset in enumerate(offset_mapping):
    answer = sample_dataset[idx]['answers']
    start_char = answer['answer_start'][0]
    end_char = start_char + len(answer['text'][0])

    # 定位答案再token中的起始位置和结束位置
    # 一种策略，我们要拿到context的起始和终止位置，然后左右逼近
    # sequence_ids(idx) 方法返回一个列表，用于指示第 idx 个示例的每个分词后的 token 属于哪个输入序列。

    context_start = tokenizer_examples.sequence_ids(idx).index(1)
    context_end = tokenizer_examples.sequence_ids(idx).index(None, context_start) - 1 # 从context_start开始查找第一个出现的None

    # 判断答案不在上下文中
    if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
        start_token_pos = 0
        end_token_pos = 0
    else:
        # 找到包含答案起始位置的上下文的开头
        token_id = context_start
        while token_id <= context_end and offset[token_id][0] < start_char:
            token_id += 1
        start_token_pos = token_id

        # 找到包含答案终止位置的上下文的结束
        token_id = context_end
        while token_id >= context_start and offset[token_id][1] > end_char:
            token_id -= 1
        end_token_pos = token_id
    
    print(answer, start_char, end_char, context_start, context_end, start_token_pos, end_token_pos)
    print("token answer decode:", tokenizer.decode(tokenizer_examples['input_ids'][idx][start_token_pos : end_token_pos + 1]))
    

In [ ]:
def process_function(examples):
    tokenizer_examples = tokenizer(text=examples['question'],
                                    text_pair=examples['context'],
                                    max_length=512,
                                    truncation='only_second',
                                    return_offsets_mapping=True,
                                    padding='max_length'
                                   )
    offset_mapping = tokenizer_examples.pop('offset_mapping')

    # 因为后面会使用到tokenizer batch的方式进行处理
    start_position = []
    end_position = []
    for idx, offset in enumerate(offset_mapping):
        answer = examples['answers'][idx]
        start_char = answer['answer_start'][0]
        end_char = start_char + len(answer['text'][0])

        # 定位答案再token中的起始位置和结束位置
        # 一种策略，我们要拿到context的起始和终止位置，然后左右逼近
        # sequence_ids(idx) 方法返回一个列表，用于指示第 idx 个示例的每个分词后的 token 属于哪个输入序列。

        context_start = tokenizer_examples.sequence_ids(idx).index(1)
        context_end = tokenizer_examples.sequence_ids(idx).index(None, context_start) - 1 # 从context_start开始查找第一个出现的None

        # 判断答案不在上下文中
        if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
            start_token_pos = 0
            end_token_pos = 0
        else:
            # 找到包含答案起始位置的上下文的开头
            token_id = context_start
            while token_id <= context_end and offset[token_id][0] < start_char:
                token_id += 1
            start_token_pos = token_id

            # 找到包含答案终止位置的上下文的结束
            token_id = context_end
            while token_id >= context_start and offset[token_id][1] > end_char:
                token_id -= 1
            end_token_pos = token_id
        start_position.append(start_token_pos)
        end_position.append(end_token_pos)
    
    tokenizer_examples['start_positions'] = start_position
    tokenizer_examples['end_positions'] = end_position
    return tokenizer_examples


In [ ]:
datasets['train']

In [ ]:
tokenizer_datasets = datasets.map(function=process_function, batched=True, remove_columns=datasets['train'].column_names)
tokenizer_datasets

## step4 创建模型

In [ ]:


model = AutoModelForQuestionAnswering.from_pretrained('D:/pretrained_model/models--hfl--chinese-macbert-base')

args = TrainingArguments(
    output_dir='./models_for_qa',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=32,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_steps=100,
    max_steps=800
)

trainer = Trainer(
    model=model,
    args = args,
    tokenizer = tokenizer,
    train_dataset= tokenizer_datasets['train'],
    eval_dataset= tokenizer_datasets['validation'],
    data_collator=DefaultDataCollator()
)

trainer.train()

## step9 模型推理

In [ ]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

pipe(question="小明在哪里上班？", context="小明在上海工作过，现在在深圳做了。")